Download audio using pytube

In [2]:
from pytube import YouTube

# Enter the URL of the YouTube video
video_url = "https://www.youtube.com/watch?v=lks-x8ZM554"

# Create a YouTube object
yt = YouTube(video_url)

# Get the audio stream
audio_stream = yt.streams.filter(only_audio=True).first()

# Download the audio stream
video_directory = "jeff dean leaders connect bangalore"
audio_stream.download(output_path=f"./{video_directory}")

'/home/analyticalmonk/open_source/local_llm_stt_summarization/jeff dean leaders connect bangalore/Leaders Connect with Jeff Dean.mp4'